In [1]:
import segmentation_models_pytorch as smp
from pathlib import Path
from pytorch_lightning import Trainer
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint #GpuUsageLoggber
from src.settings import RESULT_DIR
from src.conf.config import CONFIG
from src.dl.lightning_model import SegModel, plot_metrics
from src.dl.model_builder import SmpModelWithClsBranch, SmpGeneralModel

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/l

# Train a pytorch model

- Any model should work if it's wrapped to lightning SegModel wrapper. e.g. smp models

In [2]:
# model = smp.FPN(
#     encoder_name="resnext50_32x4d", 
#     classes=2, 
#     decoder_merge_policy='cat'
# )

In [2]:
config = CONFIG

inst_model = smp.PSPNet(
    encoder_name="resnext50_32x4d", 
    classes=2,
    # decoder_merge_policy='cat'
)

type_model = smp.PSPNet(
    encoder_name="resnext50_32x4d", 
    classes=5,
    # decoder_merge_policy='cat'
)

base_model = SmpModelWithClsBranch(inst_model, type_model)
lightning_model = SegModel.from_conf(base_model, config)

In [3]:
tt_logger = pl_loggers.TestTubeLogger(
    save_dir=RESULT_DIR,
    name=config.experiment_args.model_name,
    version=config.experiment_args.experiment_version
)

checkpoint_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
)

checkpoint_callback = ModelCheckpoint(
    filepath = str(checkpoint_dir),
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'avg_val_loss',
    mode = 'min',
    prefix = ''
)

if config.training_args.resume_training:   
    last_checkpoint_path = lightning_model.fm.model_checkpoint("last")
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        #callbacks=[GPUStatsMonitor()],
        resume_from_checkpoint=str(last_checkpoint_path),
        profiler=True,
    )
    
else:
    trainer = Trainer(
        default_root_dir=config.experiment_args.experiment_root_dir,
        max_epochs=config.training_args.num_epochs, 
        gpus=config.training_args.num_gpus,  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        # callbacks=[GpuUsageLogger()],
        profiler=True,
    )
    

# find the batch size automatically
# new_batch_size = trainer.scale_batch_size(lightning_model)

# Override old batch size
# lightning_model.batch_size = new_batch_size

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:

Checkpoint directory /home/local/leos/Dippa/results/tests/Unet/version_test_consep_PSPNet_presentation exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [4]:
log_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
    / "tf"
)

In [5]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

# Train the model

In [6]:
trainer.fit(lightning_model)

um=tion]
Epoch 9:  78%|███████▊  | 592/761 [03:01<00:51,  3.27it/s, loss=0.129, v_num=tion]
Validating: 0it [00:00, ?it/s]
Epoch 9: 100%|█████████▉| 760/761 [04:09<00:00,  3.05it/s, loss=0.129, v_num=tion]
Epoch 00009: avg_val_loss  was not in top 1
Epoch 10:  78%|███████▊  | 592/761 [03:02<00:51,  3.25it/s, loss=0.116, v_num=tion]
Validating: 0it [00:00, ?it/s]
Validating: 100%|██████████| 169/169 [01:06<00:00, 24.00it/s]
Epoch 00010: avg_val_loss  was not in top 1
Epoch 11:  78%|███████▊  | 592/761 [03:02<00:51,  3.25it/s, loss=0.139, v_num=tion]
Validating: 0it [00:00, ?it/s]
Epoch 11: 100%|█████████▉| 760/761 [04:09<00:00,  3.04it/s, loss=0.139, v_num=tion]
Epoch 00011: avg_val_loss  was not in top 1
Epoch 12:  78%|███████▊  | 592/761 [02:58<00:50,  3.32it/s, loss=0.124, v_num=tion]
Validating: 0it [00:00, ?it/s]
Epoch 12: 100%|█████████▉| 760/761 [04:03<00:00,  3.12it/s, loss=0.124, v_num=tion]
Epoch 00012: avg_val_loss  was not in top 1
Epoch 12: 100%|██████████| 761/761 [04:04<0

1

In [8]:
# plot_metrics(conf=config, metric='accuracy', scale='linear', save=True)

In [9]:
trainer.test(
    model=lightning_model,
    ckpt_path=lightning_model.fm.model_checkpoint("best").as_posix()
)

Testing:  97%|█████████▋| 37/38 [00:12<00:00,  5.82it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_accuracy': tensor(0.8935, device='cuda:0'),
 'avg_test_loss': tensor(0.2711, device='cuda:0'),
 'loss': tensor(0.2711, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 38/38 [00:12<00:00,  3.12it/s]


[{'avg_test_loss': 0.2710990011692047,
  'avg_test_accuracy': 0.8935374021530151,
  'loss': 0.2710990011692047}]